<a name="index"></a>
<h1 align="center">Análise de Dados Eleitorais</h1>
______________________________________________________________________________________________________________________________

# 1 - Introdução e Objetivo do Projeto

Este notebook tem objetivo de proporcionar um ambiente de análise de dados eleitorais.

## Objetivo das análises
* [01](#analysis_1)- Mostrar a classificação do eleitorado a partir do estado civil, escolaridade e faixa etária, além da quantidade de eleitores que utilizam nome social;  
* [02](#analysis_2)- Relacionar as características do eleitorado de acordo com o representante eleito em determinada região;  
* [03](#analysis_3)- Mostrar a renda média do eleitorado de determinada região;  
* [04](#analysis_4)- Mostrar, daquela região, a cidade com mais eleitores jovem e mais eleitores idosos;  
* [05](#analysis_5)- Comparar a renda média de eleitores do Vale do Paraíba com a de outras regiões do Estado; 
* [06](#analysis_6)- Automatizar a exportação das estatísticas;  
* [07](#analysis_7)- Relacionar características do eleitorado de acordo com o representante eleito em determinada região de eleições futuras;  
* [08](#analysis_8)- Mostrar a percentagem de eleitores ausentes da última eleição;  
* [09](#analysis_9)- Mostrar a evolução do número de eleitores;  
* [10](#analysis_10) Análise de dados feita em Jupyter Notebook.

<a name="dev"></a>
# 2 - Preparativos


Instale as dependências deste projeto através do comando:

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


Importando as libs necessarias para rodar o projeto:

In [5]:
# libs para análise de dados
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np

# libs geração de gráficos
import plotly.express as px
pd.options.plotting.backend = "plotly"
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# libs auxiliares
import os
from os import walk
import datetime

print("Importações e configurações concluídas ✔️ ")

Importações e configurações concluídas ✔️ 


# 3 - Getting started

## 3.1 Configuração: Região de Estudo

### 3.1.1 Macroregiões

Para orientar-se por macrorregiões é possível escolher um município ou uma lista de municípios. Para o exemplo iremos colocar as regiões do Vale do Paraíba, Vale Histórico, Litoral Norte, Serra da Mantiqueira e Região Brangantina:

In [18]:
macro_regiao = {
    "VALE DO PARAÍBA": [
        "APARECIDA",
        "CAÇAPAVA",
        "CACHOEIRA PAULISTA",
        "CANAS",
        "CUNHA",
        "GUARATINGUETÁ",
        "IGARATÁ",
        "JACAREÍ",
        "JAMBEIRO",
        "LAGOINHA",
        "LORENA",
        "NATIVIDADE DA SERRA",
        "PARAIBUNA",
        "PINDAMONHANGABA",
        "PIQUETE",
        "POTIM",
        "REDENÇÃO DA SERRA",
        "ROSEIRA",
        "SANTA BRANCA",
        "SÃO JOSÉ DOS CAMPOS",
        "SÃO LUÍS DO PARAITINGA",
        "TAUBATÉ",
        "TREMEMBÉ"
    ],
    "VALE HISTÓRICO": [
        "ARAPEÍ",
        "AREIAS",
        "BANANAL",
        "CRUZEIRO",
        "LAVRINHAS",
        "QUELUZ",
        "SÃO JOSÉ DO BARREIRO",
        "SILVEIRAS"
    ],
    "LITORAL NORTE": [
        "ILHABELA",
        "SÃO SEBASTIÃO",
        "UBATUBA",
        "CAMPOS DO JORDÃO",
        "MONTEIRO LOBATO",
        "SANTO ANTÔNIO DO PINHAL",
        "SÃO BENTO DO SAPUCAÍ"
    ],
    "REGIÃO BRAGANTINA": [
        "BOM JESUS DOS PERDÕES",
        "BRAGANÇA PAULISTA",
        "JOANÓPOLIS",
        "NAZARÉ PAULISTA",
        "PIRACAIA",
        "VARGEM"
    ]
}

### 3.1.2 Municípios foco

As variáveis `municipio1`, `municipio2` e `municipio3` guardarão o nome dos municípios que terão mais foco durante a análise. Tome cuidado com a escrita: o município deve ser declarado com a **acentuação e espaçamento corretos**, além de estar **entre aspas**!


In [7]:
# Informe os municípios para análise
municipio1 = "SÃO JOSÉ DOS CAMPOS"
municipio2 = "TAUBATÉ"            
municipio3 = "CAÇAPAVA"            

# Checando se você digitou corretamente
if(set([municipio1.upper(),municipio2.upper(),municipio3.upper()]).issubset(set(macroregiao.keys()))):
    print(f"Municípios OK ✔️")
else:
    print("❌ Município com nome incorreto, tente novamente!")

Municípios OK ✔️


### 3.2 Configuração: Ano eleitoral
Para análises de apenas um município, temos à disposição os dados das eleições de 2018 e 2020. Abaixo as características das eleições que ocorreram nestes anos:

ANO  | TIPO DE ELEIÇÃO    | CARGOS                  |
:----|:-------------------|:------------------------|
2018 | Federal e Estadual | Presidente e Governador |
2020 | Municipal          | Prefeito                |

A variável `eleicao_ano` guarda o ano escolhido para as análises!<br>

In [8]:
# Informe o ano para análise
eleicao_ano = 2020

# listagem de anos eleitorais para cada 'tipo'
anos_eleicoes_federal = np.arange(1994, datetime.date.today().year+4, 4).tolist()
anos_eleicoes_municipal = np.arange(1996, datetime.date.today().year+4, 4).tolist()

if eleicao_ano in anos_eleicoes_federal or eleicao_ano in anos_eleicoes_municipal:
   print(f"Ano eleitoral {eleicao_ano} escolhido com sucesso ✔️")
else:
    print("❌ O ano escolhido não é um ano de eleição! Volte ao início do notebook, altere o ano e rode aquela e esta célula novamente")

Ano eleitoral 2020 escolhido com sucesso ✔️


### 3.3 Check list Arquivos de Dados (.csv)

Os dados que utilizados podem ser obtidos a partir do [IBGE - Instituto Brasileiro de Geografia e Estatística](https://cidades.ibge.gov.br/brasil/sp/sao-jose-dos-campos/pesquisa/19/29765?localidade1=355410&localidade2=355030) e do [TSE - Tribunal Superior Eleitoral](https://www.tse.jus.br/eleicoes/estatisticas/repositorio-de-dados-eleitorais-1/repositorio-de-dados-eleitorais). <br>

**Arquivo sobre renda média dos municípios:**
- cadastro_central_de_empresas.csv

**Arquivos dos candidatos:**
- consulta_cand_2018_BRASIL.csv
- consulta_cand_2020_SP.csv

**Arquivos sobre a evolução da quantiade de eleitores:**
- eleitorado_municipio_2014.csv
- eleitorado_municipio_2016.csv
- eleitorado_municipio_2018.csv
- eleitorado_municipio_2020.csv

**Arquivos sobre comparecimento e abstenção dos eleitores:**
- perfil_comparecimento_abstencao_2018.csv
- perfil_comparecimento_abstencao_2020.csv

**Arquivos do eleitorado:**
- perfil_eleitorado_2018.csv
- perfil_eleitorado_2020.csv

### <font color='red'>**IMPORTANTE:**</font> Para ter os <i>.csv</i> mencionados disponíveis, você precisa ter todos os arquivos <i>.zip</i> descompactados dentro da pasta <b>\data</b> antes de seguir os próximos passos. 
### <font color='red'>**ATENÇÃO:**</font> As análises deste notebook são passíveis de atualização, mas para fazer análises de anos futuros é necessário colocar os novos <i>.csv</i> descompactados juntamente com os já presentes.

Para este projeto rodar é necessario ter arquivos de dados especificos no formato *.csv* na pasta */data* .
A célula a seguir analisa se todos os arquivos necessarios estão presentes de acordo com o ano escolhido na variável: `eleicao_ano`.

In [9]:
# verificando quantos CSVs com o ano escolhido existem na pasta "/data"
data_folder = 'data'

csvs = 0
for path, _, files in walk(data_folder):
    for csv in files:
        if str(eleicao_ano) in csv and '.csv' in csv:
            # caso tiver algum CSV contendo o ano escolhido, soma 1
            csvs += 1          
if csvs > 0:
     # existe pelo menos 1 CSV do ano escolhido
    if csvs < 4:
        # caso ainda faltem algum CSV, por mais que tenha pelo menos 1 com o ano escolhido
        print("Falta algum CSV para que todas as análises funcionem!")
        for path, _, files in walk(data_folder):
            for csv in files:
                if f'eleitorado_municipio_{eleicao_ano}.csv' not in csv:
                    print(f'Está faltando o arquivo: eleitorado_municipio_{eleicao_ano}.csv')
            for csv in files:
                if f'consulta_cand_{eleicao_ano}_SP.csv' not in csv:
                    print(f'Esta faltando o arquivo: consulta_cand_{eleicao_ano}_SP.csv')
            for csv in files:
                if f'consulta_cand_{eleicao_ano}_BRASIL.csv' not in csv:
                    print(f'Esta faltando o arquivo: consulta_cand_{eleicao_ano}_BRASIL.csv')
            for csv in files:
                if f'perfil_eleitorado_{eleicao_ano}.csv' not in csv:
                   print(f'Esta faltando o arquivo: perfil_eleitorado_{eleicao_ano}.csv')
            for csv in files:
                if f'perfil_comparecimento_abstencao_{eleicao_ano}.csv' not in csv:
                    print(f'Esta faltando o arquivo: perfil_comparecimento_abstencao_{eleicao_ano}.csv')
            
    else:
        # caso todos os CSVs necessários estejam na pasta
        print("O ano escolhido está disponível e com todos os CSVs necessários para as análises!")
else:
    # caso não tenha nenhum CSV do ano escolhido 
    print("Nenhum CSV disponível! Por favor, adicione os CSVs necesários na pasta '/data'!")

O ano escolhido está disponível e com todos os CSVs necessários para as análises!


### 3.4 Hands on: Carregamento dos dados

Para deixar o processo mais rápido, vamos carregar apenas as informações necessárias dentro de uma infinidade de dados que teremos acesso. <br>

In [62]:
def build_path(filename):
    return fr'./{data_folder}/{filename}'

def build_df(filename, usecols=[]):
    if usecols == []:
        return pd.read_csv(
            build_path(filename),
            sep= ';',
            on_bad_lines='skip',
            encoding='iso-8859-1')
    else:
        return pd.read_csv(
            build_path(filename),
            sep= ';',
            usecols=usecols,
            on_bad_lines='skip',
            encoding='iso-8859-1')

In [88]:
# filtro de colunas
col_candidatos = ["NR_TURNO", "NM_UE", "DS_CARGO", "NM_CANDIDATO", "SG_PARTIDO", "DS_SIT_TOT_TURNO"]
col_eleitorado = ["NR_ANO_ELEICAO", "SG_UF", "NM_MUNICIPIO", "QTD_ELEITORES"]
col_perfil_eleitorado = [
    "ANO_ELEICAO", "SG_UF", "NM_MUNICIPIO", "DS_ESTADO_CIVIL", "CD_FAIXA_ETARIA", "DS_FAIXA_ETARIA", 
    "CD_GRAU_ESCOLARIDADE", "DS_GRAU_ESCOLARIDADE", "QT_ELEITORES_PERFIL", 
    "QT_ELEITORES_INC_NM_SOCIAL", "DS_GENERO"]
col_abstencao = ["NR_TURNO", "NM_MUNICIPIO", "QT_APTOS", "QT_COMPARECIMENTO", "QT_ABSTENCAO"]
col_despesas = ["NM_UE", "DS_CARGO", "NM_CANDIDATO", "VR_DESPESA_CONTRATADA" ]

In [89]:
# Carregando os Dataframes

df_renda_mun = build_df('cadastro_central_de_empresas.csv')
df_eleitorado_2014 = build_df('eleitorado_municipio_2014.csv',col_eleitorado)
df_eleitorado_2016 = build_df('eleitorado_municipio_2016.csv',col_eleitorado)
df_eleitorado_2018 = build_df('eleitorado_municipio_2018.csv',col_eleitorado)
df_eleitorado_2020 = build_df('eleitorado_municipio_2020.csv',col_eleitorado)
df_candidatos = build_df(f'consulta_cand_{eleicao_ano}_BRASIL.csv', col_candidatos) if eleicao_ano == 2018 else build_df(f'consulta_cand_{eleicao_ano}_SP.csv', col_candidatos)
df_perfil_eleitorado = build_df(f'perfil_eleitorado_{eleicao_ano}.csv', col_perfil_eleitorado)
df_abstencao = build_df(f'perfil_comparecimento_abstencao_{eleicao_ano}.csv', col_abstencao)
df_despesas = build_df('despesas_contratadas_candidatos_2020_SP.csv', col_despesas) if eleicao_ano == 2020 else build_df('despesas_contratadas_candidatos_2018_SP.csv', col_despesas)
print('dfs carregados com sucesso!')

dfs carregados com sucesso!


E vamos filtrar o município declarado na variável `municipio1` na [seção inputs](#municipios):

In [82]:
# Filtrando o município para análises de um município isolado
municipio_eleitores = df_eleitorado.query(f'NM_MUNICIPIO == "{municipio1.upper()}"')

# 4 - Análises
<a name="analysis_1"></a>
## 4.1 - Mostrar a classificação do eleitorado a partir do estado civil, escolaridade e faixa etária, além da quantidade de eleitores que utilizam nome social 

[Back to the top](#index)


In [90]:
df_perfil_eleitorado.head(10)

,ANO_ELEICAO,SG_UF,NM_MUNICIPIO,DS_GENERO,DS_ESTADO_CIVIL,CD_FAIXA_ETARIA,DS_FAIXA_ETARIA,CD_GRAU_ESCOLARIDADE,DS_GRAU_ESCOLARIDADE,QT_ELEITORES_PERFIL,QT_ELEITORES_INC_NM_SOCIAL
0,2020,SP,SÃO PAULO,FEMININO,CASADO,9094,90 a 94 anos,6,ENSINO MÉDIO COMPLETO,17,0
1,2020,SP,SÃO PAULO,FEMININO,CASADO,9094,90 a 94 anos,7,SUPERIOR INCOMPLETO,1,0
2,2020,SP,SÃO PAULO,FEMININO,CASADO,9094,90 a 94 anos,8,SUPERIOR COMPLETO,8,0
3,2020,SP,SÃO PAULO,FEMININO,CASADO,9599,95 a 99 anos,1,ANALFABETO,28,0
4,2020,SP,SÃO PAULO,FEMININO,CASADO,9599,95 a 99 anos,2,LÊ E ESCREVE,149,0
5,2020,SP,SÃO PAULO,FEMININO,CASADO,9599,95 a 99 anos,3,ENSINO FUNDAMENTAL INCOMPLETO,157,0
6,2020,SP,SÃO PAULO,FEMININO,CASADO,9599,95 a 99 anos,4,ENSINO FUNDAMENTAL COMPLETO,74,0
7,2020,SP,SÃO PAULO,FEMININO,CASADO,9599,95 a 99 anos,6,ENSINO MÉDIO COMPLETO,10,0
8,2020,SP,SÃO PAULO,FEMININO,CASADO,9599,95 a 99 anos,8,SUPERIOR COMPLETO,1,0
9,2020,SP,SÃO PAULO,FEMININO,CASADO,9999,100 anos ou mais,1,ANALFABETO,3,0


In [92]:
genero = df_perfil_eleitorado.groupby('DS_GENERO').count()
genero

,ANO_ELEICAO,SG_UF,NM_MUNICIPIO,DS_ESTADO_CIVIL,CD_FAIXA_ETARIA,DS_FAIXA_ETARIA,CD_GRAU_ESCOLARIDADE,DS_GRAU_ESCOLARIDADE,QT_ELEITORES_PERFIL,QT_ELEITORES_INC_NM_SOCIAL
DS_GENERO,,,,,,,,,,
FEMININO,2229866,2229866,2229866,2229866,2229866,2229866,2229866,2229866,2229866,2229866
MASCULINO,1995992,1995992,1995992,1995992,1995992,1995992,1995992,1995992,1995992,1995992
NÃO INFORMADO,22236,22236,22236,22236,22236,22236,22236,22236,22236,22236


<a name="dev2"></a>
## 4.2 Candidato eleito
[Índice](#index)
<p>Qual candidato foi eleito? De qual partido ele era? Ganhou no primeiro turno?</p>
<p>Nesta análise vamos organizar os dados que mostrarão de forma parametrizada o nome do(s) candidato(s) eleito(s), o cargo, o partido eleitoral e em qual turno foi eleito, tudo isso em função do:</p>

- `eleicao_ano` escolhido na [seção inputs](#ano)
- `municipio1` escolhido na [seção inputs](#municipios), quando for ano de eleições municipais.


In [17]:
# Carrega apenas os dados de candidatos do ano da eleição escolhida pelo usuário
if eleicao_ano == 2018 or eleicao_ano == 2022 or eleicao_ano == 2026:
    presidente_candidatos = df_candidatos.query('NM_UE == "BRASIL" and DS_CARGO == "PRESIDENTE" and DS_SIT_TOT_TURNO == "ELEITO"')
    governador_candidatos = df_candidatos.query('NM_UE == "SÃO PAULO" and DS_CARGO == "GOVERNADOR" and DS_SIT_TOT_TURNO == "ELEITO"')

elif eleicao_ano == 2020 or eleicao_ano == 2024 or eleicao_ano == 2028:
    prefeito_candidatos = df_candidatos.query(f'NM_UE == "{municipio1.upper()}" and DS_CARGO == "PREFEITO" and DS_SIT_TOT_TURNO == "ELEITO"')

In [18]:
# Filtra o cargo em função do ano da eleição escolhida pelo usuário
if eleicao_ano == 2018 or eleicao_ano == 2022 or eleicao_ano == 2026:
    presidente_eleito = presidente_candidatos['NM_CANDIDATO'].iloc[0]
    presidente_eleito_partido = presidente_candidatos['SG_PARTIDO'].iloc[0]
    presidente_eleito_turno = presidente_candidatos['NR_TURNO'].iloc[0]
    governador_eleito = governador_candidatos['NM_CANDIDATO'].iloc[0]
    governador_eleito_partido = governador_candidatos['SG_PARTIDO'].iloc[0]
    governador_eleito_turno = governador_candidatos['NR_TURNO'].iloc[0]

elif eleicao_ano == 2020 or eleicao_ano == 2024 or eleicao_ano == 2028:
    prefeito_eleito = prefeito_candidatos['NM_CANDIDATO'].iloc[0]
    prefeito_eleito_partido = prefeito_candidatos['SG_PARTIDO'].iloc[0]
    prefeito_eleito_turno = prefeito_candidatos['NR_TURNO'].iloc[0]

[Resultados desta análise](#result2)

<a name="dev3"></a>
## 4.3 Análise da evolução do número de eleitores
[Índice](#index)
<p>Será que a quantidade de eleitores de um município sempre é maior a cada eleição?</p>
<p>Nesta análise vamos organizar os dados que mostrarão como a quantidade de eleitores variou entre os anos de 2014 e 2020.</p>
<p>São obtidos os históricos para:</p>

- `municipio1` escolhido na [seção inputs](#municipios)
- Cobertura da TV Vanguarda como um todo
- Macrorregiões que compõem a cobertura da TV Vanguarda

In [19]:
# Filtrando, pela estado de SP:
elei2014 = df_eleitorado2014.loc[df_eleitorado2014['SG_UF']=='SP']
elei2016 = df_eleitorado2016.loc[df_eleitorado2016['SG_UF']=='SP']
elei2018 = df_eleitorado2018.loc[df_eleitorado2018['SG_UF']=='SP']
elei2020 = df_eleitorado2020.loc[df_eleitorado2020['SG_UF']=='SP']
if eleicao_ano == 2022:
    elei2022 = df_eleitorado2022.loc[df_eleitorado2022['SG_UF']=='SP']
elif eleicao_ano == 2024:
    elei2022 = df_eleitorado2022.loc[df_eleitorado2022['SG_UF']=='SP']
    elei2024 = df_eleitorado2024.loc[df_eleitorado2024['SG_UF']=='SP']
    
# Filtrando novamente, dessa vez, pela cidade escolhida no input:
elei2014cidade = elei2014.loc[elei2014['NM_MUNICIPIO']== municipio1]
elei2016cidade = elei2016.loc[elei2016['NM_MUNICIPIO']== municipio1]
elei2018cidade = elei2018.loc[elei2018['NM_MUNICIPIO']== municipio1]
elei2020cidade = elei2020.loc[elei2020['NM_MUNICIPIO']== municipio1]
if eleicao_ano == 2022:
    elei2022cidade = elei2022.loc[elei2022['NM_MUNICIPIO']== municipio1]
elif eleicao_ano == 2024:
    elei2022cidade = elei2022.loc[elei2022['NM_MUNICIPIO']== municipio1]
    elei2024cidade = elei2024.loc[elei2024['NM_MUNICIPIO']== municipio1]

In [20]:
# Juntando os csv de cada ano em um csv apenas:
quantitativoTotal = elei2014cidade.merge(elei2016cidade, how='outer').merge(elei2018cidade, how='outer').merge(elei2020cidade, how='outer')
if eleicao_ano == 2022:
    quantitativoTotal = quantitativoTotal.merge(elei2022cidade, how='outer')
    
elif eleicao_ano == 2024:
    quantitativoTotal = quantitativoTotal.merge(elei2022cidade, how='outer')
    quantitativoTotal = quantitativoTotal.merge(elei2022cidade, how='outer').merge(elei2024cidade, how='outer')

In [21]:
# Filtro por macrorregiões da cobertura da TV Vanguarda:
macroregiao = dfmc.Município.values
vale_do_paraiba = dfmc.query('Macrorregião == "VALE DO PARAÍBA"').Município.values
vale_historico = dfmc.query('Macrorregião == "VALE HISTÓRICO"').Município.values
litoral_norte = dfmc.query('Macrorregião == "LITORAL NORTE"').Município.values
serra_da_mantiqueira = dfmc.query('Macrorregião == "SERRA DA MANTIQUEIRA"').Município.values
regiao_bragantina = dfmc.query('Macrorregião == "REGIÃO BRAGANTINA"').Município.values

In [22]:
# Criando DataFrames acumuladores para receberem os dados de cada região:
vanguarda2014 = elei2014.loc[elei2014['NM_MUNICIPIO']== tuple(macroregiao)]
vanguarda2016 = elei2016.loc[elei2016['NM_MUNICIPIO']== tuple(macroregiao)]
vanguarda2018 = elei2018.loc[elei2018['NM_MUNICIPIO']== tuple(macroregiao)]
vanguarda2020 = elei2020.loc[elei2020['NM_MUNICIPIO']== tuple(macroregiao)]

valep2014 = elei2014.loc[elei2014['NM_MUNICIPIO']== tuple(vale_do_paraiba)]
valep2016 = elei2016.loc[elei2016['NM_MUNICIPIO']== tuple(vale_do_paraiba)]
valep2018 = elei2018.loc[elei2018['NM_MUNICIPIO']== tuple(vale_do_paraiba)]
valep2020 = elei2020.loc[elei2020['NM_MUNICIPIO']== tuple(vale_do_paraiba)]

valeh2014 = elei2014.loc[elei2014['NM_MUNICIPIO']== tuple(vale_historico)]
valeh2016 = elei2016.loc[elei2016['NM_MUNICIPIO']== tuple(vale_historico)]
valeh2018 = elei2018.loc[elei2018['NM_MUNICIPIO']== tuple(vale_historico)]
valeh2020 = elei2020.loc[elei2020['NM_MUNICIPIO']== tuple(vale_historico)]

litoraln2014 = elei2014.loc[elei2014['NM_MUNICIPIO']== tuple(litoral_norte)]
litoraln2016 = elei2016.loc[elei2016['NM_MUNICIPIO']== tuple(litoral_norte)]
litoraln2018 = elei2018.loc[elei2018['NM_MUNICIPIO']== tuple(litoral_norte)]
litoraln2020 = elei2020.loc[elei2020['NM_MUNICIPIO']== tuple(litoral_norte)]

serram2014 = elei2014.loc[elei2014['NM_MUNICIPIO']== tuple(serra_da_mantiqueira)]
serram2016 = elei2016.loc[elei2016['NM_MUNICIPIO']== tuple(serra_da_mantiqueira)]
serram2018 = elei2018.loc[elei2018['NM_MUNICIPIO']== tuple(serra_da_mantiqueira)]
serram2020 = elei2020.loc[elei2020['NM_MUNICIPIO']== tuple(serra_da_mantiqueira)]

regiaob2014 = elei2014.loc[elei2014['NM_MUNICIPIO']== tuple(regiao_bragantina)]
regiaob2016 = elei2016.loc[elei2016['NM_MUNICIPIO']== tuple(regiao_bragantina)]
regiaob2018 = elei2018.loc[elei2018['NM_MUNICIPIO']== tuple(regiao_bragantina)]
regiaob2020 = elei2020.loc[elei2020['NM_MUNICIPIO']== tuple(regiao_bragantina)]

if eleicao_ano == 2022:
    vanguarda2022 = elei2022.loc[elei2022['NM_MUNICIPIO']== tuple(macroregiao)]
    valep2022 = elei2022.loc[elei2022['NM_MUNICIPIO']== tuple(vale_do_paraiba)]
    valeh2022 = elei2022.loc[elei2022['NM_MUNICIPIO']== tuple(vale_historico)]
    litoraln2022 = elei2022.loc[elei2022['NM_MUNICIPIO']== tuple(litoral_norte)]
    serram2022 = elei2022.loc[elei2022['NM_MUNICIPIO']== tuple(serra_da_mantiqueira)]
    regiaob2022 = elei2022.loc[elei2022['NM_MUNICIPIO']== tuple(regiao_bragantina)]
    
elif eleicao_ano == 2024:
    vanguarda2022 = elei2022.loc[elei2022['NM_MUNICIPIO']== tuple(macroregiao)]
    vanguarda2024 = elei2024.loc[elei2024['NM_MUNICIPIO']== tuple(macroregiao)]
    valep2022 = elei2022.loc[elei2022['NM_MUNICIPIO']== tuple(vale_do_paraiba)]
    valep2024 = elei2024.loc[elei2024['NM_MUNICIPIO']== tuple(vale_do_paraiba)]
    valeh2022 = elei2022.loc[elei2022['NM_MUNICIPIO']== tuple(vale_historico)]
    valeh2024 = elei2024.loc[elei2024['NM_MUNICIPIO']== tuple(vale_historico)]
    litoraln2022 = elei2022.loc[elei2022['NM_MUNICIPIO']== tuple(litoral_norte)]
    litoraln2024 = elei2024.loc[elei2024['NM_MUNICIPIO']== tuple(litoral_norte)]
    serram2022 = elei2022.loc[elei2022['NM_MUNICIPIO']== tuple(serra_da_mantiqueira)]
    serram2024 = elei2024.loc[elei2024['NM_MUNICIPIO']== tuple(serra_da_mantiqueira)]
    regiaob2022 = elei2022.loc[elei2022['NM_MUNICIPIO']== tuple(regiao_bragantina)]
    regiaob2024 = elei2024.loc[elei2024['NM_MUNICIPIO']== tuple(regiao_bragantina)]

In [23]:
# Preenchendo os DataFrames acumuladores com dados de cada região:
for x in macroregiao: vanguarda2014 = vanguarda2014.append(elei2014.loc[elei2014['NM_MUNICIPIO']== x] )
for x in macroregiao: vanguarda2016 = vanguarda2016.append(elei2016.loc[elei2016['NM_MUNICIPIO']== x] )
for x in macroregiao: vanguarda2018 = vanguarda2018.append(elei2018.loc[elei2018['NM_MUNICIPIO']== x] )
for x in macroregiao: vanguarda2020 = vanguarda2020.append(elei2020.loc[elei2020['NM_MUNICIPIO']== x] )

for x in vale_do_paraiba: valep2014 = valep2014.append(elei2014.loc[elei2014['NM_MUNICIPIO']== x] )
for x in vale_do_paraiba: valep2016 = valep2016.append(elei2016.loc[elei2016['NM_MUNICIPIO']== x] )
for x in vale_do_paraiba: valep2018 = valep2018.append(elei2018.loc[elei2018['NM_MUNICIPIO']== x] )
for x in vale_do_paraiba: valep2020 = valep2020.append(elei2020.loc[elei2020['NM_MUNICIPIO']== x] )

for x in vale_historico: valeh2014 = valeh2014.append(elei2014.loc[elei2014['NM_MUNICIPIO']== x] )
for x in vale_historico: valeh2016 = valeh2016.append(elei2016.loc[elei2016['NM_MUNICIPIO']== x] )
for x in vale_historico: valeh2018 = valeh2018.append(elei2018.loc[elei2018['NM_MUNICIPIO']== x] )
for x in vale_historico: valeh2020 = valeh2020.append(elei2020.loc[elei2020['NM_MUNICIPIO']== x] )

for x in litoral_norte: litoraln2014 = litoraln2014.append(elei2014.loc[elei2014['NM_MUNICIPIO']== x] )
for x in litoral_norte: litoraln2016 = litoraln2016.append(elei2016.loc[elei2016['NM_MUNICIPIO']== x] )
for x in litoral_norte: litoraln2018 = litoraln2018.append(elei2018.loc[elei2018['NM_MUNICIPIO']== x] )
for x in litoral_norte: litoraln2020 = litoraln2020.append(elei2020.loc[elei2020['NM_MUNICIPIO']== x] )

for x in serra_da_mantiqueira: serram2014 = serram2014.append(elei2014.loc[elei2014['NM_MUNICIPIO']== x] )
for x in serra_da_mantiqueira: serram2016 = serram2016.append(elei2016.loc[elei2016['NM_MUNICIPIO']== x] )
for x in serra_da_mantiqueira: serram2018 = serram2018.append(elei2018.loc[elei2018['NM_MUNICIPIO']== x] )
for x in serra_da_mantiqueira: serram2020 = serram2020.append(elei2020.loc[elei2020['NM_MUNICIPIO']== x] )

for x in regiao_bragantina: regiaob2014 = regiaob2014.append(elei2014.loc[elei2014['NM_MUNICIPIO']== x] )
for x in regiao_bragantina: regiaob2016 = regiaob2016.append(elei2016.loc[elei2016['NM_MUNICIPIO']== x] )
for x in regiao_bragantina: regiaob2018 = regiaob2018.append(elei2018.loc[elei2018['NM_MUNICIPIO']== x] )
for x in regiao_bragantina: regiaob2020 = regiaob2020.append(elei2020.loc[elei2020['NM_MUNICIPIO']== x] )
if eleicao_ano == 2022:
    for x in macroregiao: vanguarda2022 = vanguarda2022.append(elei2022.loc[elei2022['NM_MUNICIPIO']== x] )
    for x in vale_do_paraiba: valep2022 = valep2022.append(elei2022.loc[elei2022['NM_MUNICIPIO']== x] )
    for x in vale_historico: valeh2022 = valeh2022.append(elei2022.loc[elei2022['NM_MUNICIPIO']== x] )
    for x in litoral_norte: litoraln2022 = litoraln2022.append(elei2022.loc[elei2022['NM_MUNICIPIO']== x] )
    for x in serra_da_mantiqueira: serram2022 = serram2022.append(elei2022.loc[elei2022['NM_MUNICIPIO']== x] )
    for x in regiao_bragantina: regiaob2022 = regiaob2022.append(elei2022.loc[elei2022['NM_MUNICIPIO']== x] )

elif eleicao_ano == 2024:
    for x in macroregiao: vanguarda2022 = vanguarda2022.append(elei2022.loc[elei2022['NM_MUNICIPIO']== x] )
    for x in vale_do_paraiba: valep2022 = valep2022.append(elei2022.loc[elei2022['NM_MUNICIPIO']== x] )
    for x in vale_historico: valeh2022 = valeh2022.append(elei2022.loc[elei2022['NM_MUNICIPIO']== x] )
    for x in litoral_norte: litoraln2022 = litoraln2022.append(elei2022.loc[elei2022['NM_MUNICIPIO']== x] )
    for x in serra_da_mantiqueira: serram2022 = serram2022.append(elei2022.loc[elei2022['NM_MUNICIPIO']== x] )
    for x in regiao_bragantina: regiaob2022 = regiaob2022.append(elei2022.loc[elei2022['NM_MUNICIPIO']== x] )
        
    for x in macroregiao: vanguarda2024 = vanguarda2024.append(elei2024.loc[elei2024['NM_MUNICIPIO']== x] )
    for x in vale_do_paraiba: valep2024 = valep2024.append(elei2024.loc[elei2024['NM_MUNICIPIO']== x] )
    for x in vale_historico: valeh2024 = valeh2024.append(elei2024.loc[elei2024['NM_MUNICIPIO']== x] )
    for x in litoral_norte: litoraln2024 = litoraln2024.append(elei2024.loc[elei2024['NM_MUNICIPIO']== x] )
    for x in serra_da_mantiqueira: serram2024 = serram2024.append(elei2024.loc[elei2024['NM_MUNICIPIO']== x] )
    for x in regiao_bragantina: regiaob2024 = regiaob2024.append(elei2024.loc[elei2024['NM_MUNICIPIO']== x] )    

<ipython-input-23-80781f902ccc>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in macroregiao: vanguarda2014 = vanguarda2014.append(elei2014.loc[elei2014['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in macroregiao: vanguarda2014 = vanguarda2014.append(elei2014.loc[elei2014['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in macroregiao: vanguarda2014 = vanguarda2014.append(elei2014.loc[elei2014['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in

<ipython-input-23-80781f902ccc>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in macroregiao: vanguarda2016 = vanguarda2016.append(elei2016.loc[elei2016['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in macroregiao: vanguarda2016 = vanguarda2016.append(elei2016.loc[elei2016['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in macroregiao: vanguarda2016 = vanguarda2016.append(elei2016.loc[elei2016['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in

<ipython-input-23-80781f902ccc>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in macroregiao: vanguarda2018 = vanguarda2018.append(elei2018.loc[elei2018['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in macroregiao: vanguarda2018 = vanguarda2018.append(elei2018.loc[elei2018['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in macroregiao: vanguarda2018 = vanguarda2018.append(elei2018.loc[elei2018['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in

<ipython-input-23-80781f902ccc>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in macroregiao: vanguarda2020 = vanguarda2020.append(elei2020.loc[elei2020['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in macroregiao: vanguarda2020 = vanguarda2020.append(elei2020.loc[elei2020['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in macroregiao: vanguarda2020 = vanguarda2020.append(elei2020.loc[elei2020['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in

<ipython-input-23-80781f902ccc>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in vale_do_paraiba: valep2014 = valep2014.append(elei2014.loc[elei2014['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in vale_do_paraiba: valep2014 = valep2014.append(elei2014.loc[elei2014['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in vale_do_paraiba: valep2014 = valep2014.append(elei2014.loc[elei2014['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in vale_do_par

<ipython-input-23-80781f902ccc>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in vale_do_paraiba: valep2020 = valep2020.append(elei2020.loc[elei2020['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in vale_do_paraiba: valep2020 = valep2020.append(elei2020.loc[elei2020['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in vale_do_paraiba: valep2020 = valep2020.append(elei2020.loc[elei2020['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in vale_do

<ipython-input-23-80781f902ccc>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in serra_da_mantiqueira: serram2020 = serram2020.append(elei2020.loc[elei2020['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in serra_da_mantiqueira: serram2020 = serram2020.append(elei2020.loc[elei2020['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  for x in serra_da_mantiqueira: serram2020 = serram2020.append(elei2020.loc[elei2020['NM_MUNICIPIO']== x] )
<ipython-input-23-80781f902ccc>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instea

In [24]:
# Criando um novo DataFrame para cada macrorregião, a partir da soma do eleitorado de cada respectiva cidade:
# Cada dicionário abaixo representa o total de eleitores da determinada região:
dados_vanguarda = [ ["2014","COBERTURA TV VANGUARDA",int(vanguarda2014[['QTD_ELEITORES']].sum(axis=0))],
                ["2016","COBERTURA TV VANGUARDA",int(vanguarda2016[['QTD_ELEITORES']].sum(axis=0))],
                ["2018","COBERTURA TV VANGUARDA",int(vanguarda2018[['QTD_ELEITORES']].sum(axis=0))],
                ["2020","COBERTURA TV VANGUARDA",int(vanguarda2020[['QTD_ELEITORES']].sum(axis=0))] ]

dados_valep = [ ["2014","VALE DO PARAÍBA",int(valep2014[['QTD_ELEITORES']].sum(axis=0))],
                ["2016","VALE DO PARAÍBA",int(valep2016[['QTD_ELEITORES']].sum(axis=0))],
                ["2018","VALE DO PARAÍBA",int(valep2018[['QTD_ELEITORES']].sum(axis=0))],
                ["2020","VALE DO PARAÍBA",int(valep2020[['QTD_ELEITORES']].sum(axis=0))] ]

dados_valeh = [ ["2014","VALE HISTÓRICO",int(valeh2014[['QTD_ELEITORES']].sum(axis=0))],
                ["2016","VALE HISTÓRICO",int(valeh2016[['QTD_ELEITORES']].sum(axis=0))],
                ["2018","VALE HISTÓRICO",int(valeh2018[['QTD_ELEITORES']].sum(axis=0))],
                ["2020","VALE HISTÓRICO",int(valeh2020[['QTD_ELEITORES']].sum(axis=0))] ]

dados_litoraln = [ ["2014","LITORAL NORTE",int(litoraln2014[['QTD_ELEITORES']].sum(axis=0))],
                   ["2016","LITORAL NORTE",int(litoraln2016[['QTD_ELEITORES']].sum(axis=0))],
                   ["2018","LITORAL NORTE",int(litoraln2018[['QTD_ELEITORES']].sum(axis=0))],
                   ["2020","LITORAL NORTE",int(litoraln2020[['QTD_ELEITORES']].sum(axis=0))] ]

dados_serram = [ ["2014","SERRA DA MANTIQUEIRA",int(serram2014[['QTD_ELEITORES']].sum(axis=0))],
                 ["2016","SERRA DA MANTIQUEIRA",int(serram2016[['QTD_ELEITORES']].sum(axis=0))],
                 ["2018","SERRA DA MANTIQUEIRA",int(serram2018[['QTD_ELEITORES']].sum(axis=0))],
                 ["2020","SERRA DA MANTIQUEIRA",int(serram2020[['QTD_ELEITORES']].sum(axis=0))] ]

dados_regiaob = [ ["2014","REGIÃO BRAGANTINA",int(regiaob2014[['QTD_ELEITORES']].sum(axis=0))],
                  ["2016","REGIÃO BRAGANTINA",int(regiaob2016[['QTD_ELEITORES']].sum(axis=0))],
                  ["2018","REGIÃO BRAGANTINA",int(regiaob2018[['QTD_ELEITORES']].sum(axis=0))],
                  ["2020","REGIÃO BRAGANTINA",int(regiaob2020[['QTD_ELEITORES']].sum(axis=0))] ]

quantRegionalCV = pd.DataFrame(data= dados_vanguarda , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
quantRegionalVP = pd.DataFrame(data= dados_valep , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
quantRegionalVH = pd.DataFrame(data= dados_valeh , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
quantRegionalLN = pd.DataFrame(data= dados_litoraln , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
quantRegionalSM = pd.DataFrame(data= dados_serram, columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
quantRegionalRB = pd.DataFrame(data= dados_regiaob, columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])

if eleicao_ano == 2022:
    van_addano_2022 = [ ["2022","COBERTURA TV VANGUARDA",int(vanguarda2022[['QTD_ELEITORES']].sum(axis=0))] ]
    vap_addano_2022 = [["2022","VALE DO PARAÍBA",int(valep2022[['QTD_ELEITORES']].sum(axis=0))] ]
    vah_addano_2022 = [["2022","VALE HISTÓRICO",int(valeh2022[['QTD_ELEITORES']].sum(axis=0))] ]
    lit_addano_2022 =  [["2022","LITORAL NORTE",int(litoraln2022[['QTD_ELEITORES']].sum(axis=0))] ]
    ser_addano_2022 = [["2022","SERRA DA MANTIQUEIRA",int(serram2022[['QTD_ELEITORES']].sum(axis=0))] ]
    reb_addano_2022 = [["2022","REGIÃO BRAGANTINA",int(regiaob2022[['QTD_ELEITORES']].sum(axis=0))] ]
    
    van_addano_2022 = pd.DataFrame(data= van_addano_2022 , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
    vap_addano_2022 = pd.DataFrame(data= vap_addano_2022 , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
    vah_addano_2022 = pd.DataFrame(data= vah_addano_2022 , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
    lit_addano_2022 =  pd.DataFrame(data= lit_addano_2022 , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
    ser_addano_2022 = pd.DataFrame(data= ser_addano_2022 , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
    reb_addano_2022 = pd.DataFrame(data= reb_addano_2022 , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
    
    quantRegionalCV = quantRegionalCV.merge(van_addano_2022, how="outer")
    quantRegionalVP = quantRegionalVP.merge(vap_addano_2022, how="outer")
    quantRegionalVH = quantRegionalVH.merge(vah_addano_2022, how="outer")
    quantRegionalLN = quantRegionalLN.merge(lit_addano_2022, how="outer")
    quantRegionalSM = quantRegionalSM.merge(ser_addano_2022, how="outer")
    quantRegionalRB = quantRegionalRB.merge(reb_addano_2022, how="outer")

elif eleicao_ano == 2024:
    van_addano_2022 = [ ["2022","COBERTURA TV VANGUARDA",int(vanguarda2022[['QTD_ELEITORES']].sum(axis=0))] ]
    vap_addano_2022 = [["2022","VALE DO PARAÍBA",int(valep2022[['QTD_ELEITORES']].sum(axis=0))] ]
    vah_addano_2022 = [["2022","VALE HISTÓRICO",int(valeh2022[['QTD_ELEITORES']].sum(axis=0))] ]
    lit_addano_2022 =  [["2022","LITORAL NORTE",int(litoraln2022[['QTD_ELEITORES']].sum(axis=0))] ]
    ser_addano_2022 = [["2022","SERRA DA MANTIQUEIRA",int(serram2022[['QTD_ELEITORES']].sum(axis=0))] ]
    reb_addano_2022 = [["2022","REGIÃO BRAGANTINA",int(regiaob2022[['QTD_ELEITORES']].sum(axis=0))] ]
    
    van_addano_2022 = pd.DataFrame(data= van_addano_2022 , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
    vap_addano_2022 = pd.DataFrame(data= vap_addano_2022 , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
    vah_addano_2022 = pd.DataFrame(data= vah_addano_2022 , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
    lit_addano_2022 =  pd.DataFrame(data= lit_addano_2022 , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
    ser_addano_2022 = pd.DataFrame(data= ser_addano_2022 , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
    reb_addano_2022 = pd.DataFrame(data= reb_addano_2022 , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
    
    quantRegionalCV = quantRegionalCV.merge(van_addano_2022, how="outer")
    quantRegionalVP = quantRegionalVP.merge(vap_addano_2022, how="outer")
    quantRegionalVH = quantRegionalVH.merge(vah_addano_2022, how="outer")
    quantRegionalLN = quantRegionalLN.merge(lit_addano_2022, how="outer")
    quantRegionalSM = quantRegionalSM.merge(ser_addano_2022, how="outer")
    quantRegionalRB = quantRegionalRB.merge(reb_addano_2022, how="outer")

    
    van_addano_2024 = [ ["2024","COBERTURA TV VANGUARDA",int(vanguarda2024[['QTD_ELEITORES']].sum(axis=0))] ]
    vap_addano_2024 = [["2024","VALE DO PARAÍBA",int(valep2024[['QTD_ELEITORES']].sum(axis=0))] ]
    vah_addano_2024 = [["2024","VALE HISTÓRICO",int(valeh2024[['QTD_ELEITORES']].sum(axis=0))] ]
    lit_addano_2024 =  [["2024","LITORAL NORTE",int(litoraln2024[['QTD_ELEITORES']].sum(axis=0))] ]
    ser_addano_2024 = [["2024","SERRA DA MANTIQUEIRA",int(serram2024[['QTD_ELEITORES']].sum(axis=0))] ]
    reb_addano_2024 = [["2024","REGIÃO BRAGANTINA",int(regiaob2024[['QTD_ELEITORES']].sum(axis=0))] ]
    
    van_addano_2024 = pd.DataFrame(data= van_addano_2024 , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
    vap_addano_2024 = pd.DataFrame(data= vap_addano_2024 , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
    vah_addano_2024 = pd.DataFrame(data= vah_addano_2024 , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
    lit_addano_2024 =  pd.DataFrame(data= lit_addano_2024 , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
    ser_addano_2024 = pd.DataFrame(data= ser_addano_2024 , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
    reb_addano_2024 = pd.DataFrame(data= reb_addano_2024 , columns=['ANO', 'MACRO_REGIAO', 'QTD_TOTAL_ELEITORES'])
    
    quantRegionalCV = quantRegionalCV.merge(van_addano_2024, how="outer")
    quantRegionalVP = quantRegionalVP.merge(vap_addano_2024, how="outer")
    quantRegionalVH = quantRegionalVH.merge(vah_addano_2024, how="outer")
    quantRegionalLN = quantRegionalLN.merge(lit_addano_2024, how="outer")
    quantRegionalSM = quantRegionalSM.merge(ser_addano_2024, how="outer")
    quantRegionalRB = quantRegionalRB.merge(reb_addano_2024, how="outer")

[Resultados desta análise](#result3)

<a name="dev4"></a>
## 4.4 Comparecimento às urnas
[Índice](#index)
<p>Dos eleitores que estão aptos a votar numa eleição, quantos de fato compareceram às urnas e quantos não foram votar?</p>
<p>Nesta análise vamos organizar os dados que mostrarão qual foi o comparecimento e abstenção dos eleitores no 1º e 2º turno para:</p>

- `municipio1` escolhido na [seção inputs](#municipios)
- `eleicao_ano` escolhido na [seção inputs](#ano)

In [25]:
abstencao_municipio = df_abstencao.query(f'NM_MUNICIPIO == "{municipio1.upper()}"')

# Avaliando abstenções no primeiro turno
primeiro_turno_data = abstencao_municipio.query('NR_TURNO == 1')
comparecimento_primeiro_turno = primeiro_turno_data.QT_COMPARECIMENTO.sum()
abstencao_primeiro_turno = primeiro_turno_data.QT_ABSTENCAO.sum()

# Avaliando abstenções no segundo turno
segundo_turno_data = abstencao_municipio.query('NR_TURNO == 2')
comparecimento_segundo_turno = segundo_turno_data.QT_COMPARECIMENTO.sum()
abstencao_segundo_turno = segundo_turno_data.QT_ABSTENCAO.sum()

# Checa se houve ou não segundo turno
if comparecimento_segundo_turno == 0:
    print(f"obs: Não houve segundo turno na cidade de {municipio1} nas eleições de {eleicao_ano}.")

obs: Não houve segundo turno na cidade de SÃO JOSÉ DOS CAMPOS nas eleições de 2020.


In [26]:
# Calculando os valores do primeiro turno
total_eleitores_primeiro_turno = comparecimento_primeiro_turno + abstencao_primeiro_turno
comparecimento_primeiro_turno_pct = comparecimento_primeiro_turno / total_eleitores_primeiro_turno
abstencao_primeiro_turno_pct = abstencao_primeiro_turno / total_eleitores_primeiro_turno

if comparecimento_segundo_turno > 0:
    # Calculando os valores se houve segundo turno
    total_eleitores_segundo_turno = comparecimento_segundo_turno + abstencao_segundo_turno
    comparecimento_segundo_turno_pct = comparecimento_segundo_turno / total_eleitores_segundo_turno
    abstencao_segundo_turno_pct = abstencao_segundo_turno / total_eleitores_segundo_turno

[Resultados desta análise](#result4)

<a name="dev5"></a>
## 4.5 Análise dos municípios com mais eleitores jovens e  idosos
[Índice](#index)
<p>Quais municípios possuem a maior população de eleitores jovens? E de idosos?</p>
<p>Esta análise avalia quais são os municípios com a maior porcentagem de eleitores jovens e idosos para:</p> 

- `eleicao_ano` escolhido na [seção inputs](#ano)
- Cobertura da TV Vanguarda como um todo
- Macrorregiões que compõem a cobertura da TV Vanguarda

<p>Foram classificados como jovens os eleitores com idade entre 18 e 29 anos. E como idosos, eleitores com 60 anos ou mais.</p>

In [27]:
# Cria um dataframe acumulador vazio
dfm = pd.DataFrame(columns=["Macrorregião", "Município", "Jovens_pct", "Jovens_abs", "Idosos_pct", "Idosos_abs"])

# Calcula a porcentagem de eleitores jovens e idosos em cada município
for cidade in macroregiao.keys():
    filtroCidade = df_eleitorado.query("NM_MUNICIPIO == @cidade")
    filtroEleitoresTotal = filtroCidade['QT_ELEITORES_PERFIL'].sum()
    filtroEleitoresFaixaEtaria = filtroCidade.groupby(['DS_FAIXA_ETARIA'])['QT_ELEITORES_PERFIL'].sum()
   
    # Eleitores Jovens
    f18anos = (filtroEleitoresFaixaEtaria[filtroEleitoresFaixaEtaria.index.str.startswith('18 anos')]).values
    f19anos = (filtroEleitoresFaixaEtaria[filtroEleitoresFaixaEtaria.index.str.startswith('19 anos')]).values
    f20anos = (filtroEleitoresFaixaEtaria[filtroEleitoresFaixaEtaria.index.str.startswith('20 anos')]).values
    f21anos = (filtroEleitoresFaixaEtaria[filtroEleitoresFaixaEtaria.index.str.startswith('21 a 24 anos')]).values
    f25anos = (filtroEleitoresFaixaEtaria[filtroEleitoresFaixaEtaria.index.str.startswith('25 a 29 anos')]).values
    
    # Eleitores Idosos
    f60anos = (filtroEleitoresFaixaEtaria[filtroEleitoresFaixaEtaria.index.str.startswith('60 a 64 anos')]).values
    f65anos = (filtroEleitoresFaixaEtaria[filtroEleitoresFaixaEtaria.index.str.startswith('65 a 69 anos')]).values
    f70anos = (filtroEleitoresFaixaEtaria[filtroEleitoresFaixaEtaria.index.str.startswith('70 a 74 anos')]).values
    f75anos = (filtroEleitoresFaixaEtaria[filtroEleitoresFaixaEtaria.index.str.startswith('75 a 79 anos')]).values
    f80anos = (filtroEleitoresFaixaEtaria[filtroEleitoresFaixaEtaria.index.str.startswith('80 a 84 anos')]).values
    f85anos = (filtroEleitoresFaixaEtaria[filtroEleitoresFaixaEtaria.index.str.startswith('85 a 89 anos')]).values
    f90anos = (filtroEleitoresFaixaEtaria[filtroEleitoresFaixaEtaria.index.str.startswith('90 a 94 anos')]).values
    f95anos = (filtroEleitoresFaixaEtaria[filtroEleitoresFaixaEtaria.index.str.startswith('95 a 99 anos')]).values
    f100anos = (filtroEleitoresFaixaEtaria[filtroEleitoresFaixaEtaria.index.str.startswith('100 anos ou mais')]).values
    
    # Caso alguma faixa etária não exista no município analisado,
    # é atribuído o valor 0 a ela, para não atrapalhar nos cálculos.
    if len(f18anos) == 0: f18anos = [0]
    if len(f19anos) == 0: f19anos = [0]
    if len(f20anos) == 0: f20anos = [0]
    if len(f21anos) == 0: f20anos = [0]
    if len(f25anos) == 0: f20anos = [0]    
    if len(f60anos) == 0: f60anos = [0]
    if len(f65anos) == 0: f65anos = [0]
    if len(f70anos) == 0: f70anos = [0]
    if len(f75anos) == 0: f75anos = [0]
    if len(f80anos) == 0: f80anos = [0]
    if len(f85anos) == 0: f85anos = [0]
    if len(f90anos) == 0: f90anos = [0]
    if len(f95anos) == 0: f95anos = [0]
    if len(f100anos) == 0: f100anos = [0]
    
    # Removendo os "[ ]"
    f18anos = f18anos[0]
    f19anos = f19anos[0]
    f20anos = f20anos[0]
    f21anos = f21anos[0]
    f25anos = f25anos[0]
    f60anos = f60anos[0]
    f65anos = f65anos[0]
    f70anos = f70anos[0]
    f75anos = f75anos[0]
    f80anos = f80anos[0]
    f85anos = f85anos[0]
    f90anos = f90anos[0]
    f95anos = f95anos[0]
    f100anos = f100anos[0]
    
    # Output do cálculo
    abs_jovens = f18anos + f19anos + f20anos + f21anos + f25anos
    abs_idosos = f60anos + f65anos + f70anos + f75anos + f80anos + f85anos + f90anos + f95anos + f100anos
    pct_jovens = abs_jovens / filtroEleitoresTotal
    pct_idosos = abs_idosos / filtroEleitoresTotal
    
    # Gravando a cidade no dataframe acumulador
    dft = pd.DataFrame({"Macrorregião":[macroregiao[cidade]], "Município":[cidade], "Jovens_pct":[pct_jovens], "Jovens_abs":[abs_jovens], "Idosos_pct":[pct_idosos], "Idosos_abs":[abs_idosos]})
    dfm = dfm.append(dft)

AttributeError: 'numpy.ndarray' object has no attribute 'keys'

In [ ]:
# Número de muncípios que aparecerão no ranking
top = 3

# Ranking de municípios com mais jovens
topJovens = dfm[['Macrorregião','Município','Jovens_pct','Jovens_abs']]
topJovensmacroregiao = topJovens.sort_values("Jovens_pct",ascending = False).head(top)
topJovensValeParaiba = topJovens.query('Macrorregião == "VALE DO PARAÍBA"').sort_values("Jovens_pct",ascending = False).head(top)
topJovensValeHistorico = topJovens.query('Macrorregião == "VALE HISTÓRICO"').sort_values("Jovens_pct",ascending = False).head(top)
topJovensLitoralNorte = topJovens.query('Macrorregião == "LITORAL NORTE"').sort_values("Jovens_pct",ascending = False).head(top)
topJovensSerraMantiqueira = topJovens.query('Macrorregião == "SERRA DA MANTIQUEIRA"').sort_values("Jovens_pct",ascending = False).head(top)
topJovensRegiaoBragantina = topJovens.query('Macrorregião == "REGIÃO BRAGANTINA"').sort_values("Jovens_pct",ascending = False).head(top)

# Ranking de municípios com mais idosos
topIdosos = dfm[['Macrorregião','Município','Idosos_pct','Idosos_abs']]
topIdososmacroregiao = topIdosos.sort_values("Idosos_pct",ascending = False).head(top)
topIdososValeParaiba = topIdosos.query('Macrorregião == "VALE DO PARAÍBA"').sort_values("Idosos_pct",ascending = False).head(top)
topIdososValeHistorico = topIdosos.query('Macrorregião == "VALE HISTÓRICO"').sort_values("Idosos_pct",ascending = False).head(top)
topIdososLitoralNorte = topIdosos.query('Macrorregião == "LITORAL NORTE"').sort_values("Idosos_pct",ascending = False).head(top)
topIdososSerraMantiqueira = topIdosos.query('Macrorregião == "SERRA DA MANTIQUEIRA"').sort_values("Idosos_pct",ascending = False).head(top)
topIdososRegiaoBragantina = topIdosos.query('Macrorregião == "REGIÃO BRAGANTINA"').sort_values("Idosos_pct",ascending = False).head(top)

[Resultados desta análise (Jovens)](#result51) <br>
[Resultados desta análise (Idosos)](#result52)

<a name="dev6"></a>
## 4.6 Faixa etária dos eleitores do município analisado
[Índice](#index)
<p>Qual seria a distribuição de idade dos eleitores daquele município?</p>
<p>Nesta análise vamos organizar os dados que mostrarão qual é a faixa etária do eleitorado para:</p>

- `municipio1` escolhido na [seção inputs](#municipios)
- `eleicao_ano` escolhido na [seção inputs](#ano)

In [ ]:
total_eleitores_faixaetaria = municipio_eleitores.groupby('DS_FAIXA_ETARIA')['QT_ELEITORES_PERFIL'].sum()

[Resultados desta análise](#result6)

<a name="dev7"></a>
## 4.7 Grau de escolaridade dos eleitores do município analisado
[Índice](#index)
<p>Qual seria a distribuição do grau de escolaridade dos eleitores daquele município?</p>
<p>Nesta análise vamos organizar os dados que mostrarão qual é o grau de escolaridade do eleitorado para:</p>

- `municipio1` escolhido na [seção inputs](#municipios)
- `eleicao_ano` escolhido na [seção inputs](#ano)

In [ ]:
total_eleitores_escolaridade = municipio_eleitores.groupby('DS_GRAU_ESCOLARIDADE')['QT_ELEITORES_PERFIL'].sum()

[Resultados desta análise](#result7)

<a name="dev8"></a>
## 4.8 Estado civil dos eleitores do município analisado
[Índice](#index)
<p>A maioria dos eleitores são solteiros ou casados?</p>
<p>Nesta análise vamos organizar os dados que mostrarão qual é estado civil do eleitorado para:</p>

- `municipio1` escolhido na [seção inputs](#municipios)
- `eleicao_ano` escolhido na [seção inputs](#ano)

In [ ]:
total_eleitores_estadocivil = municipio_eleitores.groupby('DS_ESTADO_CIVIL')['QT_ELEITORES_PERFIL'].sum().sort_values(ascending=False)

[Resultados desta análise](#result8)

<a name="dev9"></a>
## 4.9 Quantidade de eleitores com nome social no município analisado
[Índice](#index)
<p>O nome social é um direito recente que os brasileiros conquistaram, será que tem muita gente usando nome social?</p>
<p>Nesta análise vamos organizar os dados que mostrarão qual é a quantidade de eleitores que utilizaram nome social para:</p>

- `municipio1` escolhido na [seção inputs](#municipios)
- `eleicao_ano` escolhido na [seção inputs](#ano)

In [ ]:
total_eleitores_nomesocial = 0 # Iniciando a variável
df_nomesocial = municipio_eleitores.groupby('QT_ELEITORES_INC_NM_SOCIAL')['QT_ELEITORES_INC_NM_SOCIAL'].sum()

# Somando a quantidade de eleitores com nome social em cada grupo
for index_nomesocial in range(len(df_nomesocial)):
    total_eleitores_nomesocial += df_nomesocial.index[index_nomesocial] * df_nomesocial.values[index_nomesocial]

[Resultados desta análise](#result9)

<a name="dev10"></a>
## 4.10 Total de Gastos dos Candidatos por Eleitor
[Índice](#index)
<p>Qual o valor total das despesas de cada Candidato nas eleições?</p>
<p>Dessas despesas, qual o valor gasto por eleitor que cada canditato investiu em sua candidatura?</p>

<p>Análise feita de acordo com as variáveis:</p>

- `municipio1` escolhido na [seção inputs](#municipios)
- `eleicao_ano` escolhido na [seção inputs](#ano)

In [ ]:
# Selecionando um Cargo a partir do ano da eleição
if eleicao_ano == 2020:
    cargo = 'Prefeito'
elif eleicao_ano == 2018:
    cargo = 'Governador'
    municipio1 = 'São Paulo'

In [ ]:
# Criando o DataFrame
df_gastos_eleitores = (df_despesas.query(f'NM_UE == "{municipio1.upper()}"')).query(f'DS_CARGO == "{cargo}"')
df_gastos_eleitores['VR_DESPESA_CONTRATADA'] = df_gastos_eleitores['VR_DESPESA_CONTRATADA'].apply(lambda x:str(x).replace(",","."))
df_gastos_eleitores['VR_DESPESA_CONTRATADA'] = df_gastos_eleitores['VR_DESPESA_CONTRATADA'].astype(float)
df_gastos_eleitores =df_gastos_eleitores.drop(columns=["NM_UE", "DS_CARGO"])
total = df_gastos_eleitores.groupby('NM_CANDIDATO')['VR_DESPESA_CONTRATADA'].sum()
df_total = pd.DataFrame({'Candidatos':total.index, 'Gastos':total.values})

In [ ]:
# Levantamento da Quantidade de eleitores e adicionando-os ao DataFrame
if eleicao_ano == 2020:
    df_total['Eleitores'] = quantitativoTotal.iat[3,3]
elif eleicao_ano == 2018:
    df_total['Eleitores'] = 33037983
elif eleicao_ano == 2016:
    df_total['Eleitores'] = quantitativoTotal.iat[1,3]
elif eleicao_ano == 2014:
    df_total['Eleitores'] = 31998432

In [ ]:
# Adicionando nova coluna com o valor per capita de eleitores
df_total['Gastos_por_Eleitor'] = df_total['Gastos']/df_total['Eleitores']

[Resultados desta análise](#result10)

<a name="result"></a>
# 5 Resultados

<a name="result1"></a>
## 5.1 Análise da renda média da população
[Índice](#index)<br>
[Link para o código desta análise](#dev1)

In [ ]:
fig=comparacao.set_index('Localidade').T.plot()
fig.update_layout(title='SALÁRIO MÉDIO MENSAL DOS MUNICÍPIOS ESCOLHIDOS',
                 xaxis=dict(title='Ano'), yaxis=dict(title='Salários Mínimos'))
fig.show()

In [ ]:
fig=comparacao_regioes.set_index('Localidade').T.plot()
fig.update_layout(title='SALÁRIO MÉDIO MENSAL DAS MACRORREGIÕES DE COBERTURA DA TV VANGUARDA',
                 xaxis=dict(title='Ano'), yaxis=dict(title='Salários Mínimos'))
fig.show()

In [ ]:
fig=comparacao_cidades.set_index('Localidade').T.plot()
fig.update_layout(title='COMPARAÇÃO DO SALÁRIO MÉDIO MENSAL DOS MUNICÍPIOS ESCOLHIDOS E MACRORREGIÕES',
                 xaxis=dict(title='Ano'), yaxis=dict(title='Salários Mínimos'))
fig.show()

#### Tabela com salários mínimos:

In [ ]:
comparacao_cidades

<a name="result2"></a>
## 5.2 Candidato eleito
[Índice](#index)<br>
[Link para o código desta análise](#dev2)

In [ ]:
if eleicao_ano == 2018 or eleicao_ano == 2022 or eleicao_ano == 2026:
    print(f"Em {eleicao_ano}, foi eleito(a) para Presidente da República o(a) candidato(a) {presidente_eleito} ({presidente_eleito_partido}), no {presidente_eleito_turno}º turno das eleições.\n")
    print(f"Neste mesmo ano, foi eleito(a) para Governador(a) do estado de SP o(a) candidato(a) {governador_eleito} ({governador_eleito_partido}), no {governador_eleito_turno}º turno das eleições.")

elif eleicao_ano == 2020 or eleicao_ano == 2024 or eleicao_ano == 2028:
    print(f"Em {eleicao_ano}, foi eleito(a) para prefeito(a) da cidade de {municipio1} o(a) candidato(a) {prefeito_eleito}, do partido {prefeito_eleito_partido}, no {prefeito_eleito_turno}º turno.")

<a name="result3"></a>
## 5.3 Análise da evolução do número de eleitores
[Índice](#index)<br>
[Link para o código desta análise](#dev3)

In [ ]:
# Plot do gráfico da quantidade de eleitores pela cidade escolhida pelo usuário
graficoMunicipal = px.bar(quantitativoTotal, x='NR_ANO_ELEICAO', y='QTD_ELEITORES',
             hover_data=['QTD_ELEITORES'],
             labels={'QTD_ELEITORES':'Quantidade de Eleitores', 'NR_ANO_ELEICAO': 'Ano' },
             title = 'Evolução do eleitorado do municípios de ' + municipio1, width=720)

graficoMunicipal.show()

In [ ]:
# Plot do gráfico da quantidade de eleitores pelas macrorregiões
fig1 = px.bar(quantRegionalCV, x='ANO', y='QTD_TOTAL_ELEITORES')
fig2 = px.bar(quantRegionalVP, x='ANO', y='QTD_TOTAL_ELEITORES')
fig3 = px.bar(quantRegionalVH, x='ANO', y='QTD_TOTAL_ELEITORES')
fig4 = px.bar(quantRegionalLN, x='ANO', y='QTD_TOTAL_ELEITORES')
fig5 = px.bar(quantRegionalSM, x='ANO', y='QTD_TOTAL_ELEITORES')
fig6 = px.bar(quantRegionalRB, x='ANO', y='QTD_TOTAL_ELEITORES')

fig = make_subplots(rows=3, cols=2, subplot_titles=("Cobertura TV Vanguarda", "Vale do Paraíba", "Vale Histórico", "Litoral Norte", "Serra da Mantiqueira", "Região Bragantina"))

fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig2['data'][0], row=1, col=2)
fig.add_trace(fig3['data'][0], row=2, col=1)
fig.add_trace(fig4['data'][0], row=2, col=2)
fig.add_trace(fig5['data'][0], row=3, col=1)
fig.add_trace(fig6['data'][0], row=3, col=2)

fig.update_layout(height=1000, width=1000, title_text="Evolução do eleitorado na cobertura da TV e Vanguarda e suas Macrorregiões")

fig.show()

<a name="result4"></a>
## 5.4 Comparecimento às urnas
[Índice](#index)<br>
[Link para o código desta análise](#dev4)

In [ ]:
if comparecimento_segundo_turno == 0:
    # Avisando que só teve o primeiro turno
    print(f'Houve apenas o primeiro turno, com um total de {total_eleitores_primeiro_turno} eleitores aptos a votar.')

    # Gerando gráfico caso tenha tido apenas o primeiro turno
    labels=['Comparecimento', 'Abstenção']
    fig = go.Figure([go.Bar(
        x=labels, 
        y=[comparecimento_primeiro_turno, abstencao_primeiro_turno], 
        hovertext=[f'{comparecimento_primeiro_turno_pct*100:.1f}%', f'{abstencao_primeiro_turno_pct*100:.1f}%'])])
    fig.update_layout(title="COMPARECIMENTO ÀS URNAS - " + municipio1.upper() + " / " + str(eleicao_ano))
    fig.show()

elif comparecimento_segundo_turno > 0:
    # Avisando total de eleitores aptos a votar em cada turno
    print(f'Eleitores aptos a votar no primeiro turno: {total_eleitores_primeiro_turno}')
    print(f'Eleitores aptos a votar no segundo turno: {total_eleitores_segundo_turno}')

    # Gerando o gráfico caso tenha tido segundo turno
    labels=['Comparecimento', 'Abstenção']
    fig = go.Figure(data=[
        go.Bar(
            name='Primeiro Turno', 
            x=labels, 
            y=[comparecimento_primeiro_turno, abstencao_primeiro_turno], 
            hovertext=[f'{comparecimento_primeiro_turno_pct*100:.1f}%', f'{abstencao_primeiro_turno_pct*100:.1f}%']),
        go.Bar(
            name='Segundo Turno', 
            x=labels, 
            y=[comparecimento_segundo_turno, abstencao_segundo_turno], 
            hovertext=[f'{comparecimento_segundo_turno_pct*100:.1f}%', f'{abstencao_segundo_turno_pct*100:.1f}%'])
    ])
    fig.update_layout(title="COMPARECIMENTO ÀS URNAS - " + municipio1.upper() + " / " + str(eleicao_ano),
        barmode='group')
    fig.show()

<a name="result5"></a>
## 5.5 Análise dos municípios com mais eleitores jovens e  idosos
<a name="result51"></a>
### 5.5.1 Jovens
[Índice](#index)<br>
[Link para o código desta análise](#dev5)

In [ ]:
# Gráfico Jovens Cobertura Vanguarda
fig1 = go.Figure(data=[go.Bar(x = topJovensmacroregiao['Município'],
                              y = topJovensmacroregiao['Jovens_abs'], name = 'Eleitores',
                               text= round(topJovensmacroregiao['Jovens_pct']*100, 2), textposition="auto")])

fig2 = go.Figure(data=[go.Bar(x = topJovensValeParaiba['Município'],
                              y = topJovensValeParaiba['Jovens_abs'], name = 'Eleitores',
                              text= round(topJovensValeParaiba['Jovens_pct']*100, 2), textposition="auto")])
      
fig3 = go.Figure(data=[go.Bar(x = topJovensValeHistorico['Município'],
                              y = topJovensValeHistorico['Jovens_abs'], name = 'Eleitores',
                              text= round(topJovensValeHistorico['Jovens_pct']*100, 2), textposition="auto")])

fig4 = go.Figure(data=[go.Bar(x = topJovensLitoralNorte['Município'],
                              y = topJovensLitoralNorte['Jovens_abs'], name = 'Eleitores',
                              text= round(topJovensLitoralNorte['Jovens_pct']*100, 2), textposition="auto")])
          
fig5 = go.Figure(data=[go.Bar(x = topJovensSerraMantiqueira['Município'],
                              y = topJovensSerraMantiqueira['Jovens_abs'], name = 'Eleitores',
                              text= round(topJovensSerraMantiqueira['Jovens_pct']*100, 2), textposition="auto")])

fig6 = go.Figure(data=[go.Bar(x = topJovensRegiaoBragantina['Município'],
                              y = topJovensRegiaoBragantina['Jovens_abs'], name = 'Eleitores',
                              text= round(topJovensRegiaoBragantina['Jovens_pct']*100, 2), textposition="auto")])
                 
trace1 = fig1['data'][0]
trace2 = fig2['data'][0]
trace3 = fig3['data'][0]
trace4 = fig4['data'][0]
trace5 = fig5['data'][0]
trace6 = fig6['data'][0]
          
          
fig = make_subplots(rows=3, cols=2, subplot_titles=(
    "Cobertura TV Vanguarda [%, abs]",
    "Vale do Paraíba [%, abs]", 
    "Vale Histórico [%, abs]", 
    "Litoral Norte [%, abs]",
    "Serra da Mantiqueira [%, abs]",
    "Região Bragantina [%, abs]"))

fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=1, col=2)
fig.add_trace(trace3, row=2, col=1)
fig.add_trace(trace4, row=2, col=2)
fig.add_trace(trace5, row=3, col=1)
fig.add_trace(trace6, row=3, col=2)


# Atualizar título e altura

fig.update_layout(height=1100, width=1000,
                  title_text="Municípios da Cobertura da TV Vanguarda com mais eleitores Jovens em " + str(eleicao_ano))

fig . update_xaxes (tickangle  =  10)
fig .update_layout ( showlegend = False )

#### Tabela dos municípios com maior eleitorado jovem:

In [ ]:
topJovens.sort_values("Jovens_pct",ascending = False).head(10)

<a name="result52"></a>
### 5.5.2 Idosos
[Índice](#index)<br>
[Link para o código desta análise](#dev5)

In [ ]:
fig1 = go.Figure(data=[go.Bar(x = topIdososmacroregiao['Município'],
                              y = topIdososmacroregiao['Idosos_abs'],
                               text= round(topIdososmacroregiao['Idosos_pct']*100, 2), textposition="auto")])

fig2 = go.Figure(data=[go.Bar(x = topIdososValeParaiba['Município'],
                              y = topIdososValeParaiba['Idosos_abs'],
                              text= round(topIdososValeParaiba['Idosos_pct']*100, 2), textposition="auto")])
      
fig3 = go.Figure(data=[go.Bar(x = topIdososValeHistorico['Município'],
                              y = topIdososValeHistorico['Idosos_abs'],
                              text= round(topIdososValeHistorico['Idosos_pct']*100, 2), textposition="auto")])

fig4 = go.Figure(data=[go.Bar(x = topIdososLitoralNorte['Município'],
                              y = topIdososLitoralNorte['Idosos_abs'],
                              text= round(topIdososLitoralNorte['Idosos_pct']*100, 2), textposition="auto")])
          
fig5 = go.Figure(data=[go.Bar(x = topIdososSerraMantiqueira['Município'],
                              y = topIdososSerraMantiqueira['Idosos_abs'],
                              text= round(topIdososSerraMantiqueira['Idosos_pct']*100, 2), textposition="auto")])

fig6 = go.Figure(data=[go.Bar(x = topIdososRegiaoBragantina['Município'],
                              y = topIdososRegiaoBragantina['Idosos_abs'],
                              text= round(topIdososRegiaoBragantina['Idosos_pct']*100, 2), textposition="auto")])
                 
trace1 = fig1['data'][0]
trace2 = fig2['data'][0]
trace3 = fig3['data'][0]
trace4 = fig4['data'][0]
trace5 = fig5['data'][0]
trace6 = fig6['data'][0]
          
          
fig = make_subplots(rows=3, cols=2, subplot_titles=(
    "Cobertura TV Vanguarda [%, abs]",
    "Vale do Paraíba [%, abs]", 
    "Vale Histórico [%, abs]", 
    "Litoral Norte [%, abs]",
    "Serra da Mantiqueira [%, abs]",
    "Região Bragantina [%, abs]"))

fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=1, col=2)
fig.add_trace(trace3, row=2, col=1)
fig.add_trace(trace4, row=2, col=2)
fig.add_trace(trace5, row=3, col=1)
fig.add_trace(trace6, row=3, col=2)


# Atualizar título e altura

fig.update_layout(height=1100, width=1040,
                  title_text="Municípios da Cobertura da TV Vanguarda com mais eleitores Idosos em " + str(eleicao_ano))
                  
fig.update_xaxes (tickangle  =  10)
fig .update_layout ( showlegend = False )

#### Tabela dos municípios com maior eleitorado idoso:

In [ ]:
topIdosos.sort_values("Idosos_pct",ascending = False).head(10)

<a name="result6"></a>
## 5.6 Faixa etária dos eleitores do município analisado
[Índice](#index)<br>
[Link para o código desta análise](#dev6)

In [ ]:
fig=px.bar(total_eleitores_faixaetaria, x= total_eleitores_faixaetaria.index, y=total_eleitores_faixaetaria.values,
          title="FAIXA ETÁRIA DO ELEITORADO - " + municipio1.upper() + " / " + str(eleicao_ano), 
          labels={'y':'Eleitores', 'DS_FAIXA_ETARIA':'Faixa Etária'})

fig.show()

<a name="result7"></a>
## 5.7 Grau de escolaridade dos eleitores do município analisado
[Índice](#index)<br>
[Link para o código desta análise](#dev7)

In [ ]:
fig=px.bar(total_eleitores_escolaridade, x= total_eleitores_escolaridade.index, y=total_eleitores_escolaridade.values,
          title="GRAU DE ESCOLARIDADE DO ELEITORADO - " + municipio1.upper() + " / " + str(eleicao_ano), 
          labels={'y':'Eleitores', 'DS_GRAU_ESCOLARIDADE':'Escolaridade'})

fig.show()

<a name="result8"></a>
## 5.8 Estado civil dos eleitores do município analisado
[Índice](#index)<br>
[Link para o código desta análise](#dev8)

In [ ]:
fig=px.bar(total_eleitores_estadocivil, x= total_eleitores_estadocivil.index, y=total_eleitores_estadocivil.values,
          title="ESTADO CIVIL DO ELEITORADO - " + municipio1.upper() + " / " + str(eleicao_ano), 
          labels={'y':'Eleitores', 'DS_ESTADO_CIVIL':'Estado Civil'})

fig.show()

<a name="result9"></a>
## 5.9 Quantidade de eleitores com nome social no município analisado
[Índice](#index)<br>
[Link para o código desta análise](#dev9)
<p>A adoção do nome social e o reconhecimento da identidade de gênero foi garantido pelo Decreto Nº8.727, divulgado em abril de 2016 pela Presidência da República. Ele prevê que a pessoa transgênero ou travesti tem o direito de solicitar a inclusão do seu nome social em documentos oficiais e nos registros dos sistemas de informação dos órgãos e entidades da administração pública federal.</p>
<p>Abaixo podemos ver quantas pessoas declararam o uso de nome social no município que estamos analisando:</p>

In [ ]:
# Exibindo resultado na tela
if total_eleitores_nomesocial > 1:
    print(f"Em {municipio1}, estavam inscritos {total_eleitores_nomesocial} eleitores com nome social no ano de {eleicao_ano}.")
elif total_eleitores_nomesocial == 1:
    print(f"Em {municipio1}, estava inscrito {total_eleitores_nomesocial} eleitor com nome social no ano de {eleicao_ano}.")
else:
    print(f"Em {municipio1}, não estava inscrito nenhum eleitor com nome social no ano de {eleicao_ano}.")

<a name="result10"></a>
## 5.10 Total de gastos dos Candidatos por eleitor
[Índice](#index)<br>
[Link para o código desta análise](#dev10)
<p>Analise dos gastos dos candidatos nas eleições em razão do eleitorado.</p>
<p>Sendo que 2020 a análise será Municicpal para Prefeito e 2018, Estadual para Governador</p>
<p>Abaixo podemos ver:</p>

In [ ]:
fig = go.Figure(data=[go.Bar(x= df_total['Candidatos'], y=df_total['Gastos'],
                              text= round(df_total['Gastos_por_Eleitor'], 2), textposition="auto")])

fig.update_layout(title="GASTOS POR ELEITOR DOS CANDITATOS EM R$ - " + municipio1.upper() + " / " + str(eleicao_ano),
                  yaxis=dict(title="Gastos em R$"), xaxis=dict(title="Candidatos") )
fig.update_layout(height=600, width=1000)
fig.show()

In [ ]:
df_total

[Resultados desta análise](#dev10)

# 6. Exportar resultados das análises

Caso você queira ter acesso aos resultados das análises que realizou, nesta seção você poderá configurar o notebook para exportá-los no formato *.csv*, e assim poderá manuseá-los em outras ferramentas.

Será exportado pelo menos um arquivo *.csv* para cada análise realizada neste notebook. Todos irão conter a [numeração](#index) da respectiva análise e também uma breve descrição dela no nome do arquivo.

#### <font color='red'>IMPORTANTE: </font>Para ligar a opção de exportar os arquivos .csv, você deve declarar a variável `exportar_csv = 1`, e para desligar declare `exportar_csv = 0`. Na sequência, você deve rodar o notebook inteiro para que as últimas modificações apareçam nos arquivos exportados.

Os arquivos poderão ser encontrados no mesmo diretório onde este notebook está localizado, dentro da pasta */exportedcsv*.

In [ ]:
# Ligar ou desligar a opção de exportar resultados das análises como .csv 
exportar_csv = 0     # [0 - NÃO / 1 - SIM]
export_dir = r'./exportedcsv'

In [ ]:
# Gera o .csv de cada análise

if exportar_csv == 1:
    # 0. Checa se a pasta onde serão exportados os .csv existe, caso não, ela é criada
    if os.path.isdir(export_dir) == False:
        os.mkdir(export_dir)
    
    
    # 1. Análise da renda média da população
    comparacao_cidades.to_csv(export_dir+'/01_AnaliseRendaMediaPopulacao.csv', sep=';', encoding='iso-8859-1', index=False)
    
    
    # 2. Candidato eleito
    if eleicao_ano == 2018:
        df_candidato = presidente_candidatos.append(governador_candidatos, ignore_index=True)
        df_candidato.rename(columns={'NR_TURNO': 'TURNO', 'NM_UE': 'UF', 'DS_CARGO': 'CARGO', 'NM_CANDIDATO': 'CANDIDATO(A)', 'SG_PARTIDO': 'PARTIDO', 'DS_SIT_TOT_TURNO': 'RESULTADO'}, inplace=True)
        df_candidato['ANO'] = eleicao_ano
        df_candidato = df_candidato[['ANO', 'TURNO', 'UF', 'CARGO', 'CANDIDATO(A)', 'PARTIDO', 'RESULTADO']]
    
    elif eleicao_ano == 2020:
        df_candidato = prefeito_candidatos
        df_candidato.rename(columns={'NR_TURNO': 'TURNO', 'NM_UE' : 'MUNICÍPIO', 'DS_CARGO': 'CARGO', 'NM_CANDIDATO': 'CANDIDATO(A)', 'SG_PARTIDO': 'PARTIDO', 'DS_SIT_TOT_TURNO': 'RESULTADO'}, inplace=True)
        df_candidato['ANO'] = eleicao_ano
        df_candidato = df_candidato[['ANO', 'TURNO', 'MUNICÍPIO', 'CARGO', 'CANDIDATO(A)', 'PARTIDO', 'RESULTADO']]
    
    df_candidato.to_csv(export_dir+'/02_CandidatoEleito.csv', sep=';', encoding='iso-8859-1', index=False)
    
    
    # 3. Análise da evolução do número de eleitores
    df_quantSJC = quantitativoTotal.drop('SG_UF', axis=1)
    df_quantSJC.rename(columns={'NR_ANO_ELEICAO':'ANO', 'NM_MUNICIPIO':'MACRO_REGIAO', 'QTD_ELEITORES':'QTD_TOTAL_ELEITORES'}, inplace=True)
    df_evolucaoeleitores = df_quantSJC.append([quantRegionalCV, quantRegionalVP, quantRegionalVH, quantRegionalLN, quantRegionalSM, quantRegionalRB])
    df_evolucaoeleitores.rename(columns={'MACRO_REGIAO':'MUNICÍPIO/MACRORREGIÃO', 'QTD_TOTAL_ELEITORES':'TOTAL DE ELEITORES'}, inplace=True)
    df_evolucaoeleitores.to_csv(export_dir+'/03_AnaliseEvolucaoEleitores.csv', sep=';', encoding='iso-8859-1', index=False)
    
    
    # 4. Comparecimento às urnas
    df_comparecimento = pd.DataFrame({'ANO' : [eleicao_ano],
                                      'MUNICÍPIO' : [municipio1.upper()],
                                      'TOTAL ELEITORES 1º TURNO' : [total_eleitores_primeiro_turno],
                                      'COMPARECIMENTO 1º TURNO [ABS]' : [comparecimento_primeiro_turno],
                                      'COMPARECIMENTO 1º TURNO [%]' : [comparecimento_primeiro_turno / total_eleitores_primeiro_turno],
                                      'ABSTENÇÕES 1º TURNO [ABS]' : [abstencao_primeiro_turno],
                                      'ABSTENÇÕES 1º TURNO [%]' : [abstencao_primeiro_turno / total_eleitores_primeiro_turno]})
    
    if comparecimento_segundo_turno > 0:
        df_comparecimento2 = pd.DataFrame({'TOTAL ELEITORES 2º TURNO' : [total_eleitores_segundo_turno],
                                           'COMPARECIMENTO 2º TURNO [ABS]' : [comparecimento_segundo_turno],
                                           'COMPARECIMENTO 2º TURNO [%]' : [comparecimento_segundo_turno / total_eleitores_segundo_turno],
                                           'ABSTENÇÕES 2º TURNO [ABS]' : [abstencao_segundo_turno],
                                           'ABSTENÇÕES 2º TURNO [%]' : [abstencao_segundo_turno / total_eleitores_segundo_turno]})
        df_comparecimento = pd.concat([df_comparecimento, df_comparecimento2], axis=1)
    
    df_comparecimento.to_csv(export_dir+'/04_ComparecimentoUrnas.csv', sep=';', encoding='iso-8859-1', index=False)
    
    
    # 5. Análise dos municípios com mais eleitores jovens e idosos
    df_jovens = topJovensValeParaiba.append([topJovensValeHistorico, topJovensLitoralNorte, topJovensSerraMantiqueira, topJovensRegiaoBragantina])
    df_idosos = topIdososValeParaiba.append([topIdososValeHistorico, topIdososLitoralNorte, topIdososSerraMantiqueira, topIdososRegiaoBragantina])
    
    topJovens.rename(columns={'Macrorregião':'MACRORREGIÃO', 'Município':'MUNICÍPIO', 'Jovens_pct':'JOVENS [%]', 'Jovens_abs':'JOVENS [ABS]'}, inplace=True)
    topIdosos.rename(columns={'Macrorregião':'MACRORREGIÃO', 'Município':'MUNICÍPIO', 'Idosos_pct':'IDOSOS [%]', 'Idosos_abs':'IDOSOS [ABS]'}, inplace=True)
    df_jovens.rename(columns={'Macrorregião':'MACRORREGIÃO', 'Município':'MUNICÍPIO', 'Jovens_pct':'JOVENS [%]', 'Jovens_abs':'JOVENS [ABS]'}, inplace=True)
    df_idosos.rename(columns={'Macrorregião':'MACRORREGIÃO', 'Município':'MUNICÍPIO', 'Idosos_pct':'IDOSOS [%]', 'Idosos_abs':'IDOSOS [ABS]'}, inplace=True)
    
    topJovens.to_csv(export_dir+'/05-1_TotalEleitoresJovensPorMunicipio.csv', sep=';', encoding='iso-8859-1', index=False)
    topIdosos.to_csv(export_dir+'/05-2_TotalEleitoresIdososPorMunicipio.csv', sep=';', encoding='iso-8859-1', index=False)
    df_jovens.to_csv(export_dir+'/05-3_MunicipiosMaiorEleitoradoJovem.csv', sep=';', encoding='iso-8859-1', index=False)
    df_idosos.to_csv(export_dir+'/05-4_MunicipiosMaiorEleitoradoIdoso.csv', sep=';', encoding='iso-8859-1', index=False)
    
    
    # 6. Faixa etária dos eleitores do município analisado
    df_faixaetaria = total_eleitores_faixaetaria.to_frame()
    df_faixaetaria['FAIXA ETÁRIA'] = df_faixaetaria.index
    df_faixaetaria = df_faixaetaria.reset_index(drop=True)
    df_faixaetaria['MUNICÍPIO'] = municipio1.upper()
    df_faixaetaria['ANO ELEIÇÃO'] = eleicao_ano
    df_faixaetaria.rename(columns={'QT_ELEITORES_PERFIL':'QUANTIDADE DE ELEITORES'}, inplace=True)
    df_faixaetaria[df_faixaetaria.columns[::-1]].to_csv(export_dir+'/06_FaixasEtariasEleitorado.csv', sep=';', encoding='iso-8859-1', index=False)
    
    
    # 7. Grau de escolaridade dos eleitores do município analisado
    df_escolaridade = total_eleitores_escolaridade.to_frame()
    df_escolaridade['GRAU DE ESCOLARIDADE'] = df_escolaridade.index
    df_escolaridade = df_escolaridade.reset_index(drop=True)
    df_escolaridade['MUNICÍPIO'] = municipio1.upper()
    df_escolaridade['ANO ELEIÇÃO'] = eleicao_ano
    df_escolaridade.rename(columns={'QT_ELEITORES_PERFIL':'QUANTIDADE DE ELEITORES'}, inplace=True)
    df_escolaridade[df_escolaridade.columns[::-1]].to_csv(export_dir+'/07_EscolaridadeEleitorado.csv', sep=';', encoding='iso-8859-1', index=False)
    
    
    # 8. Estado civil dos eleitores do município analisado
    df_estadocivil = total_eleitores_estadocivil.to_frame()
    df_estadocivil['ESTADO CIVIL'] = df_estadocivil.index
    df_estadocivil = df_estadocivil.reset_index(drop=True)
    df_estadocivil['MUNICÍPIO'] = municipio1.upper()
    df_estadocivil['ANO ELEIÇÃO'] = eleicao_ano
    df_estadocivil.rename(columns={'QT_ELEITORES_PERFIL':'QUANTIDADE DE ELEITORES'}, inplace=True)
    df_estadocivil[df_estadocivil.columns[::-1]].to_csv(export_dir+'/08_EstadoCivilEleitorado.csv', sep=';', encoding='iso-8859-1', index=False)
    
    
    # 9. Quantidade de eleitores com nome social no município analisado
    df_nomesocial = pd.DataFrame({'ELEITORES COM NOME SOCIAL' : [total_eleitores_nomesocial]})
    df_nomesocial['MUNICÍPIO'] = municipio1.upper()    
    df_nomesocial['ANO ELEIÇÃO'] = eleicao_ano
    df_nomesocial[df_nomesocial.columns[::-1]].to_csv(export_dir+'/09_EleitoresNomeSocial.csv', sep=';', encoding='iso-8859-1', index=False)
    
    # 10. Gastos do Candidato por eleitor do município analisado
    df_total['Gastos'] = round(df_total['Gastos'], 2)
    df_total['Gastos_por_Eleitor'] = round(df_total['Gastos_por_Eleitor'], 2)
    df_total.rename(columns={'Gastos_por_Eleitor':'Gasto por Eleitor (milhões)'}, inplace=True)
    df_total.to_csv(export_dir+'/10_GastosCandidadosPorEleitor.csv', sep=';', encoding='iso-8859-1', index=False)


# E agora?
Bom, esses foram os dados que pudemos analisar, e agora que já sabe o que pode fazer e como fazer, fique a vontade para analisar os dados de outros municípios e anos, mas claro, que estejam dentro do nosso alcance. Para isso, consulte as tabelas e listas que estão no começo deste notebook ✨